<a href="https://colab.research.google.com/github/adamDucken/adamDucken/blob/main/cpi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fredapi
!pip install lightgbm
!pip install xgboost
!pip install catboost

In [ ]:
import lightgbm as lgb
import numpy as np, pandas as pd
import sklearn
from fredapi import Fred

In [ ]:
fred_key = '51455cd20c3cc922caededa08ef3d816'
fred = Fred(api_key=fred_key)

In [ ]:
def fred_to_df(series_arr:list, grpby_freq:str=None, grpby_method:str=None):
    """
    Takes as argument list of series_id, Example : ['GDP', 'FEDFUNDS'],
    All series should be the same frequency if they are not specify
    groupby freq, method for more info check pandas.DataFrame.resample,
    Follow Fred API docs for more info about series, their names, etc,
    This function returns pd.DataFrame() containing all series
    Example usage: series_arr = ['FEDFUNDS', 'CPIAUCSL', 'UNRATE', 'GDP']
    df = fred_to_df(series_arr, grpby_freq='Y', grpby_method='mean')
    """
    df = pd.DataFrame()
    freq = []
    import warnings

    for series_id in series_arr:
        # Get series from Fred and add to df
        df_series = pd.DataFrame(fred.get_series(series_id=series_id), columns=[series_id])
        df = pd.concat([df, df_series], axis=1)

        # Get info about series and extract frequency
        series_info = fred.get_series_info(series_id)
        freq.append(series_info['frequency'])

    same = all(i == freq[0] for i in freq)

    if not same and grpby_freq is None:
        raise Warning("Your series data is not the same frequency, please provide groupby frequency and groupby method")
        return None

    df.reset_index(inplace=True)
    df.rename(columns = {'index':'DATE'},inplace=True)
    if not same:
        df.set_index('DATE', inplace=True)
        df = df.resample(grpby_freq).apply(grpby_method)
        df.reset_index(inplace=True)
    return df

In [ ]:
series_arr = ['CPIAUCSL','MICH','PPIACO','DTB3','DGS1','DFF','DGS20','DPRIME','PI','DSPI','CPIENGSL','USALOLITONOSTSAM','PCEPILFE','UNRATE']

In [ ]:
fred_df= fred_to_df(series_arr, grpby_freq='M', grpby_method='mean')
df = fred_df.copy()

In [ ]:
df = df.set_index('DATE')
target_col = ['CPIAUCSL','PCEPILFE','CPIENGSL','PPIACO']

for col in target_col:
    for lag in range(1, 2):  # creating 3 months of lagged features as an example
        df[f'{col}_lag{lag}'] = df[col].shift(lag)
df = df.drop(columns = target_col)
df

,MICH,DTB3,DGS1,DFF,DGS20,DPRIME,PI,DSPI,USALOLITONOSTSAM,UNRATE,CPIAUCSL_lag1,PCEPILFE_lag1,CPIENGSL_lag1,PPIACO_lag1
DATE,,,,,,,,,,,,,,
1913-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1913-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.100
1913-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.000
1913-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.000
1913-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-31,2.9,5.239000,4.992500,5.33,4.463000,8.5,23746.6,20779.7,NaN,3.8,311.054,121.224,284.175,254.926
2024-04-30,3.2,5.241364,5.135000,5.33,4.767727,8.5,23809.6,20831.7,NaN,3.9,312.230,121.629,287.399,254.963
2024-05-31,3.3,5.252727,5.159545,5.33,4.711364,8.5,23923.7,20925.6,NaN,4.0,313.207,121.944,290.631,256.772


In [ ]:
df['CPI_PI'] = df['CPIAUCSL_lag1'] / df['PI']
df['CPI_DSPI'] = df['CPIAUCSL_lag1'] / df['DSPI']
df['UNRATE_PI'] = df['UNRATE'] / df['PI']
df['UNRATE_DSPI'] = df['UNRATE'] / df['DSPI']
df['DGS20_CPI'] = df['DGS20'] / df['CPIAUCSL_lag1']
df['DFF_CPI'] = df['DFF'] / df['CPIAUCSL_lag1']
df['CPIENGSL_CPI'] = df['CPIENGSL_lag1'] / df['CPIAUCSL_lag1']
df['PI_PCEPILFE'] = df['PI'] / df['PCEPILFE_lag1']
df['DSPI_PCEPILFE'] = df['DSPI'] / df['PCEPILFE_lag1']
df['DGS20_DGS1'] = df['DGS20'] - df['DGS1']
df['DGS1_DTB3'] = df['DGS1'] - df['DTB3']

In [ ]:
df = df.dropna()

In [ ]:
pd.set_option('display.max_columns', None)
df

,MICH,DTB3,DGS1,DFF,DGS20,DPRIME,PI,DSPI,USALOLITONOSTSAM,UNRATE,CPIAUCSL_lag1,PCEPILFE_lag1,CPIENGSL_lag1,PPIACO_lag1,CPI_PI,CPI_DSPI,UNRATE_PI,UNRATE_DSPI,DGS20_CPI,DFF_CPI,CPIENGSL_CPI,PI_PCEPILFE,DSPI_PCEPILFE,DGS20_DGS1,DGS1_DTB3
DATE,,,,,,,,,,,,,,,,,,,,,,,,,
1978-01-31,5.2,6.440476,7.283333,6.703548,8.139524,7.931818,1760.1,1547.4,101.41070,6.4,62.300,31.227,51.600,66.200,0.035396,0.040261,0.003636,0.004136,0.130650,0.107601,0.828250,56.364684,49.553271,0.856190,0.842857
1978-02-28,6.4,6.445556,7.337778,6.784286,8.216111,8.000000,1776.2,1563.8,101.44650,6.3,62.700,31.414,51.100,66.800,0.035300,0.040095,0.003547,0.004029,0.131038,0.108202,0.814992,56.541669,49.780353,0.878333,0.892222
1978-03-31,6.3,6.293636,7.310455,6.793226,8.206364,8.000000,1797.9,1587.1,101.55220,6.3,63.000,31.535,50.600,67.500,0.035041,0.039695,0.003504,0.003970,0.130260,0.107829,0.803175,57.012843,50.328207,0.895909,1.016818
1978-04-30,6.7,6.286000,7.454500,6.888667,8.324000,8.000000,1821.9,1604.6,101.70130,6.1,63.400,31.706,51.000,68.100,0.034799,0.039511,0.003348,0.003802,0.131293,0.108654,0.804416,57.462310,50.608718,0.869500,1.168500
1978-05-31,6.9,6.408095,7.820000,7.360000,8.441429,8.250000,1837.0,1615.1,101.85210,6.0,63.900,31.911,51.400,69.000,0.034785,0.039564,0.003266,0.003715,0.132104,0.115180,0.804382,57.566356,50.612641,0.621429,1.411905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-30,3.2,5.319500,5.436000,5.330000,4.653000,8.500000,23176.2,20392.5,99.48235,3.8,306.187,119.449,288.392,257.680,0.013211,0.015015,0.000164,0.000186,0.015197,0.017408,0.941882,194.025902,170.721396,-0.783000,0.116500
2023-10-31,4.2,5.335238,5.424762,5.330000,5.129524,8.500000,23189.4,20403.8,99.53435,3.8,307.288,119.842,291.710,258.934,0.013251,0.015060,0.000164,0.000186,0.016693,0.017345,0.949305,193.499775,170.255837,-0.295238,0.089524
2023-11-30,4.5,5.270000,5.281429,5.330000,4.842381,8.500000,23241.5,20449.8,99.60859,3.7,307.531,120.015,285.488,255.192,0.013232,0.015038,0.000159,0.000181,0.015746,0.017332,0.928323,193.654960,170.393701,-0.439048,0.011429


In [ ]:
percentage_columns = ['MICH','UNRATE', 'DTB3', 'DGS1', 'DFF', 'DGS20', 'DPRIME']

# Apply percentage change to non-percentage columns
df_pct_change = df.copy()
for col in df.columns:
    if col in percentage_columns:
        # Apply first difference
        df_pct_change[col] = df[col].diff()
    elif col != 'DATE':
        # Apply percentage change
        df_pct_change[col] = df[col].pct_change()

# Drop the first row because it will contain NaN values after pct_change and diff
df_pct_change.dropna(inplace=True)

In [ ]:
X = df_pct_change
df = X
df.reset_index(inplace=True)
df['DATE'] = pd.to_datetime(df['DATE'])

# Set 'DATE' as the index
df = df.set_index('DATE')

# Format the date to 'yyyy-mm' and set it back as index
df.index = df.index.to_period('M')

# If you want to keep 'DATE' as a column formatted as 'yyyy-mm'
df['DATE'] = df.index.to_timestamp().strftime('%Y-%m')
df = df.drop(columns = ['DATE'])
X = df
X

,MICH,DTB3,DGS1,DFF,DGS20,DPRIME,PI,DSPI,USALOLITONOSTSAM,UNRATE,CPIAUCSL_lag1,PCEPILFE_lag1,CPIENGSL_lag1,PPIACO_lag1,CPI_PI,CPI_DSPI,UNRATE_PI,UNRATE_DSPI,DGS20_CPI,DFF_CPI,CPIENGSL_CPI,PI_PCEPILFE,DSPI_PCEPILFE,DGS20_DGS1,DGS1_DTB3
DATE,,,,,,,,,,,,,,,,,,,,,,,,,
1978-02,1.2,0.005079,0.054444,0.080737,0.076587,0.068182,0.009147,0.010598,0.000353,-0.1,0.006421,0.005988,-0.009690,0.009063,-0.002702,-0.004134,-0.024548,-0.025948,0.002970,0.005588,-0.016008,0.003140,0.004583,0.025862,0.058569
1978-03,-0.1,-0.151919,-0.027323,0.008940,-0.009747,0.000000,0.012217,0.014900,0.001042,0.0,0.004785,0.003852,-0.009785,0.010479,-0.007343,-0.009966,-0.012070,-0.014681,-0.005943,-0.003450,-0.014500,0.008333,0.011005,0.020010,0.139647
1978-04,0.4,-0.007636,0.144045,0.095441,0.117636,0.000000,0.013349,0.011026,0.001468,-0.2,0.006349,0.005423,0.007905,0.008889,-0.006907,-0.004626,-0.044501,-0.042306,0.007935,0.007652,0.001546,0.007884,0.005574,-0.029477,0.149173
1978-05,0.2,0.122095,0.365500,0.471333,0.117429,0.250000,0.008288,0.006544,0.001483,-0.1,0.007886,0.006466,0.007843,0.013216,-0.000398,0.001334,-0.024479,-0.022788,0.006172,0.060061,-0.000043,0.001811,0.000078,-0.285304,0.208305
1978-06,-0.4,0.325996,0.274545,0.236667,0.085390,0.386364,0.010234,0.006563,0.001119,-0.1,0.009390,0.005891,0.005837,0.007246,-0.000836,0.002808,-0.026628,-0.023078,0.000719,0.022554,-0.003520,0.004317,0.000668,-0.304389,-0.036440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09,-0.3,0.022978,0.068609,0.000000,0.196478,0.000000,0.003538,0.003252,0.000623,0.0,0.005118,0.000980,0.043568,0.015148,0.001575,0.001860,-0.003525,-0.003241,0.038772,-0.005092,0.038254,0.002555,0.002269,-0.140382,0.643865
2023-10,1.0,0.015738,-0.011238,0.000000,0.476524,0.000000,0.000570,0.000554,0.000523,0.0,0.003596,0.003290,0.011505,0.004867,0.003025,0.003040,-0.000569,-0.000554,0.098462,-0.003583,0.007881,-0.002712,-0.002727,-0.622940,-0.231555
2023-11,0.3,-0.065238,-0.143333,0.000000,-0.287143,0.000000,0.002247,0.002254,0.000746,-0.1,0.000791,0.001444,-0.021329,-0.014452,-0.001453,-0.001460,-0.028498,-0.028506,-0.056724,-0.000790,-0.022103,0.000802,0.000810,0.487097,-0.872340


In [ ]:
series = ['CPIAUCSL']
fred_df = fred_to_df(series_arr =series, grpby_freq='ME', grpby_method='mean')

In [ ]:
y = fred_df

In [ ]:
df= y
df['DATE'] = pd.to_datetime(df['DATE'])

# Set 'DATE' as the index
df = df.set_index('DATE')

# Format the date to 'yyyy-mm' and set it back as index
df.index = df.index.to_period('M')

# If you want to keep 'DATE' as a column formatted as 'yyyy-mm'
df['DATE'] = df.index.to_timestamp().strftime('%Y-%m')
df = df.drop(columns = ['DATE'])
y = df

In [ ]:
y = y.pct_change()
y.dropna(inplace= True)

In [ ]:
X.index.dtype, y.index.dtype

(period[M], period[M])

In [ ]:
X.index = X.index.to_timestamp()
y.index = y.index.to_timestamp()

# Ensure that the indices match by aligning y to X
y_aligned = y.loc[X.index]

# Verify the alignment
print(f"Shape of y: {y_aligned.shape}")
print(f"Shape of X: {X.shape}")

# Concatenate the DataFrames along columns (axis=1)
combined_df = pd.concat([y_aligned, X], axis=1)

Shape of y: (471, 1)
Shape of X: (471, 25)


In [ ]:
# combined_df.drop(columns=['level_0','index'],inplace=True)

In [ ]:
data = combined_df
data

,CPIAUCSL,MICH,DTB3,DGS1,DFF,DGS20,DPRIME,PI,DSPI,USALOLITONOSTSAM,UNRATE,CPIAUCSL_lag1,PCEPILFE_lag1,CPIENGSL_lag1,PPIACO_lag1,CPI_PI,CPI_DSPI,UNRATE_PI,UNRATE_DSPI,DGS20_CPI,DFF_CPI,CPIENGSL_CPI,PI_PCEPILFE,DSPI_PCEPILFE,DGS20_DGS1,DGS1_DTB3
DATE,,,,,,,,,,,,,,,,,,,,,,,,,,
1978-02-01,0.004785,1.2,0.005079,0.054444,0.080737,0.076587,0.068182,0.009147,0.010598,0.000353,-0.1,0.006421,0.005988,-0.009690,0.009063,-0.002702,-0.004134,-0.024548,-0.025948,0.002970,0.005588,-0.016008,0.003140,0.004583,0.025862,0.058569
1978-03-01,0.006349,-0.1,-0.151919,-0.027323,0.008940,-0.009747,0.000000,0.012217,0.014900,0.001042,0.0,0.004785,0.003852,-0.009785,0.010479,-0.007343,-0.009966,-0.012070,-0.014681,-0.005943,-0.003450,-0.014500,0.008333,0.011005,0.020010,0.139647
1978-04-01,0.007886,0.4,-0.007636,0.144045,0.095441,0.117636,0.000000,0.013349,0.011026,0.001468,-0.2,0.006349,0.005423,0.007905,0.008889,-0.006907,-0.004626,-0.044501,-0.042306,0.007935,0.007652,0.001546,0.007884,0.005574,-0.029477,0.149173
1978-05-01,0.009390,0.2,0.122095,0.365500,0.471333,0.117429,0.250000,0.008288,0.006544,0.001483,-0.1,0.007886,0.006466,0.007843,0.013216,-0.000398,0.001334,-0.024479,-0.022788,0.006172,0.060061,-0.000043,0.001811,0.000078,-0.285304,0.208305
1978-06-01,0.007752,-0.4,0.325996,0.274545,0.236667,0.085390,0.386364,0.010234,0.006563,0.001119,-0.1,0.009390,0.005891,0.005837,0.007246,-0.000836,0.002808,-0.026628,-0.023078,0.000719,0.022554,-0.003520,0.004317,0.000668,-0.304389,-0.036440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-01,0.003596,-0.3,0.022978,0.068609,0.000000,0.196478,0.000000,0.003538,0.003252,0.000623,0.0,0.005118,0.000980,0.043568,0.015148,0.001575,0.001860,-0.003525,-0.003241,0.038772,-0.005092,0.038254,0.002555,0.002269,-0.140382,0.643865
2023-10-01,0.000791,1.0,0.015738,-0.011238,0.000000,0.476524,0.000000,0.000570,0.000554,0.000523,0.0,0.003596,0.003290,0.011505,0.004867,0.003025,0.003040,-0.000569,-0.000554,0.098462,-0.003583,0.007881,-0.002712,-0.002727,-0.622940,-0.231555
2023-11-01,0.001603,0.3,-0.065238,-0.143333,0.000000,-0.287143,0.000000,0.002247,0.002254,0.000746,-0.1,0.000791,0.001444,-0.021329,-0.014452,-0.001453,-0.001460,-0.028498,-0.028506,-0.056724,-0.000790,-0.022103,0.000802,0.000810,0.487097,-0.872340


In [ ]:
X = data.iloc[:,1:]

In [ ]:
y = data.iloc[:,:1]

In [ ]:
def y_to_bin(y):
    y['CPIAUCSL'] = y['CPIAUCSL'].apply(lambda x: 2 if x > 0 else (0 if x < 0 else 1))
    return y

In [ ]:
y = y_to_bin(y)

<ipython-input-109-9cb8eaa62495>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['CPIAUCSL'] = y['CPIAUCSL'].apply(lambda x: 2 if x > 0 else (0 if x < 0 else 1))


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((120, 4), (30, 4), (120,), (30,))

In [ ]:
# from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, BaggingClassifier,
#                               GradientBoostingClassifier, ExtraTreesClassifier, StackingClassifier,
#                               VotingClassifier)
# from sklearn.linear_model import LogisticRegression
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.svm import SVC
# from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
# from sklearn.neural_network import MLPClassifier
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

# from catboost import CatBoostClassifier
# from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier

# # Define models
# models = [
#     RandomForestClassifier(),
#     AdaBoostClassifier(),
#     BaggingClassifier(),
#     GradientBoostingClassifier(),
#     ExtraTreesClassifier(),
#     LogisticRegression(),
#     KNeighborsClassifier(),
#     DecisionTreeClassifier(),
#     SVC(probability=True),
#     GaussianNB(),
#     BernoulliNB(),
#     MLPClassifier(),
#     LinearDiscriminantAnalysis(),
#     QuadraticDiscriminantAnalysis(),
#     # CatBoostClassifier(),
#     XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
#     LGBMClassifier()
# ]

# # Define parameter grids
# param_grids = [
#     {'n_estimators': [100, 200], 'max_depth': [10, 20, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]},
#     {'n_estimators': [50, 100], 'learning_rate': [1.0, 0.1, 0.01]},
#     {'n_estimators': [10, 20], 'max_samples': [0.5, 1.0], 'max_features': [0.5, 1.0]},
#     {'n_estimators': [100, 200], 'learning_rate': [0.1, 0.01], 'max_depth': [3, 5, 7]},
#     {'n_estimators': [100, 200], 'max_depth': [10, 20, None]},
#     {'C': [0.1, 1, 10], 'solver': ['lbfgs', 'liblinear']},
#     {'n_neighbors': [3, 5, 7], 'weights': ['uniform', 'distance']},
#     {'max_depth': [None, 10, 20], 'min_samples_split': [2, 5, 10]},
#     {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf'], 'gamma': ['scale', 'auto']},
#     {},  # GaussianNB has no hyperparameters
#     {'alpha': [0.1, 1, 10]},
#     {'hidden_layer_sizes': [(50,), (100,), (50, 50)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']},
#     {},  # LinearDiscriminantAnalysis has few hyperparameters
#     {},  # QuadraticDiscriminantAnalysis has few hyperparameters
#     # {'iterations': [100, 200], 'depth': [4, 6, 10], 'learning_rate': [0.01, 0.1]},
#     {'n_estimators': [100, 200], 'max_depth': [3, 6, 10], 'learning_rate': [0.01, 0.1]},
#     {'num_leaves': [31, 50], 'max_depth': [-1, 10, 20], 'n_estimators': [100, 200, 500]}
# ]

# # Create ModelTrainer instance
# trainer = ModelTrainer(models=models, param_grids=param_grids, scoring='accuracy')

# # Fit models
# trainer.fit(X, y)

# # Get results
# results_df = trainer.get_results()

In [ ]:
# results_df = results_df.sort_values(by='f1_score',ascending= False).reset_index()

In [ ]:
# results_df

In [ ]:
# results_df.best_params.iloc[0]

In [ ]:
# results_df.model.iloc[0]

In [ ]:
# from sklearn.metrics import classification_report, accuracy_score, log_loss
# from sklearn.model_selection import GridSearchCV
# from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, BaggingClassifier,
#                               GradientBoostingClassifier, ExtraTreesClassifier)
# from sklearn.linear_model import LogisticRegression
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.svm import SVC
# from sklearn.naive_bayes import GaussianNB, BernoulliNB
# from sklearn.neural_network import MLPClassifier
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
# from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
# from catboost import CatBoostClassifier

# # Mapping from string names to model classes
# model_mapping = {
#     'RandomForestClassifier': RandomForestClassifier,
#     'AdaBoostClassifier': AdaBoostClassifier,
#     'BaggingClassifier': BaggingClassifier,
#     'GradientBoostingClassifier': GradientBoostingClassifier,
#     'ExtraTreesClassifier': ExtraTreesClassifier,
#     'LogisticRegression': LogisticRegression,
#     'KNeighborsClassifier': KNeighborsClassifier,
#     'DecisionTreeClassifier': DecisionTreeClassifier,
#     'SVC': SVC,
#     'GaussianNB': GaussianNB,
#     'BernoulliNB': BernoulliNB,
#     'MLPClassifier': MLPClassifier,
#     'LinearDiscriminantAnalysis': LinearDiscriminantAnalysis,
#     'QuadraticDiscriminantAnalysis': QuadraticDiscriminantAnalysis,
#     'XGBClassifier': XGBClassifier,
#     'LGBMClassifier': LGBMClassifier,
#     'CatBoostClassifier': CatBoostClassifier
# }

# def evaluate_model(model_class, param_grid, X_train, y_train, X_test, y_test):
#     # Initialize GridSearchCV with cross-validation
#     grid_search = GridSearchCV(estimator=model_class(), param_grid=param_grid,
#                                cv=5, n_jobs=-1, scoring='accuracy', verbose=1)

#     # Fit the model
#     grid_search.fit(X_train, y_train)

#     # Get the best model
#     best_model = grid_search.best_estimator_

#     # Make predictions
#     y_pred = best_model.predict(X_test)

#     # Make probability predictions
#     y_prob = best_model.predict_proba(X_test)

#     # Evaluate the model
#     accuracy = accuracy_score(y_test, y_pred)
#     logloss = log_loss(y_test, y_prob)
#     report = classification_report(y_test, y_pred, target_names=['-1', '0', '1'])

#     # Print results
#     print(f"Best parameters: {grid_search.best_params_}")
#     print(f"Accuracy: {accuracy}")
#     print(f"Log Loss: {logloss}")
#     print(f"Classification Report:\n{report}")

#     return {
#         "model": best_model,
#         "best_params": grid_search.best_params_,
#         "accuracy": accuracy,
#         "log_loss": logloss,
#         "classification_report": report
#     }

# # Example usage with top 5 models
# top_5_results = results_df.head(5)
# evaluations = []

# for index, row in top_5_results.iterrows():
#     model_class_name = row['model']
#     best_params = row['best_params']

#     # Get the actual model class from the mapping
#     model_class = model_mapping[model_class_name]

#     # Create a param grid with single values
#     param_grid = {k: [v] for k, v in best_params.items()}

#     # Evaluate the model
#     evaluation = evaluate_model(model_class, param_grid, X_train, y_train, X_test, y_test)
#     evaluations.append(evaluation)


In [ ]:
!pip install optuna

In [ ]:
import numpy as np
import optuna
import lightgbm as lgb
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import pandas as pd
from optuna.visualization import (
    plot_optimization_history,
    plot_param_importances,
    plot_parallel_coordinate,
)
#parameter range
l1_start, l1_end = 1e-8, 10.0
l2_start, l2_end = 1e-8, 10.0
num_leaves_start, num_leaves_end = 2, 256
feature_fraction_start, feature_fraction_end = 0.4, 1.0
bagging_fraction_start, bagging_fraction_end = 0.4, 1.0
bagging_freq_start, bagging_freq_end = 1, 7
min_child_samples_start, min_child_samples_end = 5, 100


def objective(trial):
    train_x, valid_x, train_y, valid_y = X_train, X_test, y_train, y_test
    dtrain = lgb.Dataset(train_x, label=train_y)

    param = {
        "objective": "multiclass",
        "metric": "multi_logloss",
        "num_class": 3,
        "device_type": "cpu",
        "verbosity": -1,
        "is_unbalance": True,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", l1_start, l1_end, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", l2_start, l2_end, log=True),
        "num_leaves": trial.suggest_int("num_leaves", num_leaves_start, num_leaves_end),
        "feature_fraction": trial.suggest_float("feature_fraction",feature_fraction_start, feature_fraction_end),
        "bagging_fraction": trial.suggest_float("bagging_fraction", bagging_fraction_start, bagging_fraction_end),
        "bagging_freq": trial.suggest_int("bagging_freq",bagging_freq_start, bagging_freq_end),
        "min_child_samples": trial.suggest_int("min_child_samples",min_child_samples_start, min_child_samples_end),
    }

    gbm = lgb.train(param, dtrain)
    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)
    logloss = sklearn.metrics.log_loss(valid_y, pred_labels)
    return logloss


if __name__ == "__main__":

    trial_results = []

    epochs = 1000
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    study = optuna.create_study(direction="minimize", sampler = optuna.samplers.GPSampler())
    study.optimize(objective, n_trials=epochs, show_progress_bar=True)


    for trial in study.trials:
        trial_data = {
            'Trial Number': trial.number,
            'Value': trial.value
        }
        # Add individual parameter columns
        trial_data.update(trial.params)
        trial_results.append(trial_data)

    # Convert to DataFrame
    df_trials = pd.DataFrame(trial_results)

    # Display the results
    print("Number of finished trials: {}".format(len(study.trials)))
    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    # Save the DataFrame to a CSV file
    df_trials.to_csv('second_study.csv', index=False)
    plot_optimization_history(study).show()
    plot_param_importances(study).show()
    plot_parallel_coordinate(study).show()

<ipython-input-139-f1e07c0b50f7>:57: ExperimentalWarning: GPSampler is experimental (supported from v3.6.0). The interface can change in the future.
  study = optuna.create_study(direction="minimize", sampler = optuna.samplers.GPSampler())


  0%|          | 0/1000 [00:00<?, ?it/s]

Number of finished trials: 1000
Best trial:
  Value: 2.2880056065618004
  Params: 
    lambda_l1: 4.4374987905342065e-08
    lambda_l2: 1e-08
    num_leaves: 2
    feature_fraction: 0.9434419287256459
    bagging_fraction: 0.7911081671115741
    bagging_freq: 5
    min_child_samples: 44


In [ ]:
first_study = pd.read_csv('/content/second_study.csv')

In [ ]:
first_study = first_study.sort_values(by='Value').copy()
first_study_best = first_study.iloc[:int(len(first_study)/5)]
first_study_best.describe()

,Trial Number,Value,lambda_l1,lambda_l2,num_leaves,feature_fraction,bagging_fraction,bagging_freq,min_child_samples
count,20.00000,20.000000,2.000000e+01,2.000000e+01,20.000000,20.000000,20.000000,20.000000,20.000000
mean,54.50000,2.701306,1.335652e-01,8.482776e-01,94.800000,0.736589,0.654002,4.900000,24.200000
std,27.13029,0.114688,5.052383e-01,2.129436e+00,65.337665,0.078099,0.127897,1.618967,19.462785
min,2.00000,2.655848,5.098774e-08,1.068715e-07,2.000000,0.634022,0.430053,2.000000,7.000000
25%,31.75000,2.655848,2.908842e-07,2.584400e-02,46.250000,0.689497,0.587362,3.000000,11.750000
50%,63.00000,2.661630,1.590129e-06,9.280465e-02,73.000000,0.726394,0.675016,6.000000,17.000000
75%,73.75000,2.678977,7.197267e-03,5.016059e-01,139.000000,0.754536,0.709639,6.000000,25.250000
max,90.00000,3.035255,2.267795e+00,9.522872e+00,245.000000,0.921461,0.980704,7.000000,74.000000


In [ ]:
best_trial = study.best_trial

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, accuracy_score, log_loss, classification_report
import lightgbm as lgb

params = best_trial.params

model = lgb.LGBMClassifier(**params, verbose=-1)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)

recall = recall_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
accuracy = accuracy_score(y_test, y_pred)
logloss = log_loss(y_test, y_prob)

print(f"Log Loss: {logloss}")
print(f"Accuracy: {accuracy}")
print(f"Macro Recall: {recall}")
print(f"Macro Precision: {precision}")
print(f"Macro F1-score: {f1}")
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=['0', '1', '2']))

Log Loss: 0.2852734602400593
Accuracy: 0.9263157894736842
Macro Recall: 0.9263157894736842
Macro Precision: 0.9108695652173913
Macro F1-score: 0.9052847836077095
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.38      0.55         8
           1       0.00      0.00      0.00         2
           2       0.92      1.00      0.96        85

    accuracy                           0.93        95
   macro avg       0.64      0.46      0.50        95
weighted avg       0.91      0.93      0.91        95



/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined 

In [ ]:
# from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, log_loss, classification_report
# import lightgbm as lgb

# # Best trial parameters from your research
# params_for_test = {
#     "lambda_l1": 0.000021,
#     "lambda_l2": 1.994707,
#     "num_leaves": 495,
#     "feature_fraction": 0.867928,
#     "bagging_fraction": 0.415804,
#     "bagging_freq": 9,
#     "min_child_samples": 34,
#     "objective": "multiclass",
#     "metric": "multi_logloss",
#     "num_class": 3,
#     "device_type": "cpu",
#     "verbosity": -1,
#     "is_unbalance": True,
#     "boosting_type": "gbdt"
# }

# # Combine static and best trial parameters
# params = {**params_for_test}

# # Initialize the model with the combined parameters
# model = lgb.LGBMClassifier(**params)

# # Fit the model
# model.fit(X_train, y_train)

# # Make predictions
# y_pred = model.predict(X_test)
# y_prob = model.predict_proba(X_test)

# # Evaluate the model
# recall = recall_score(y_test, y_pred, average='macro')
# precision = precision_score(y_test, y_pred, average='macro')
# f1 = f1_score(y_test, y_pred, average='macro')
# accuracy = accuracy_score(y_test, y_pred)
# logloss = log_loss(y_test, y_prob)

# # Print evaluation metrics
# print(f"Log Loss: {logloss}")
# print(f"Accuracy: {accuracy}")
# print(f"Macro Recall: {recall}")
# print(f"Macro Precision: {precision}")
# print(f"Macro F1-score: {f1}")
# print("Classification Report:")
# print(classification_report(y_test, y_pred, target_names=['0', '1', '2']))

In [ ]:
"""
Best model
(1.8970343889009034,
 {'lambda_l1': 1.5252340292891545e-08,
  'lambda_l2': 2.649415789229553e-08,
  'num_leaves': 256,
  'feature_fraction': 0.8676374022267322,
  'bagging_fraction': 0.718409562107019,
  'bagging_freq': 6,
  'min_child_samples': 50})

Macro Recall: 0.6666666666666666
Macro Precision: 0.9814814814814815
Macro F1-score: 0.7682539682539682
Classification Report:
              precision    recall  f1-score   support

    Decrease       1.00      0.50      0.67         8
   Stay Flat       1.00      0.50      0.67         2
    Increase       0.94      1.00      0.97        85

    accuracy                           0.95        95
   macro avg       0.98      0.67      0.77        95
weighted avg       0.95      0.95      0.94        95
"""

"\nBest model\n(1.8970343889009034,\n {'lambda_l1': 1.5252340292891545e-08,\n  'lambda_l2': 2.649415789229553e-08,\n  'num_leaves': 256,\n  'feature_fraction': 0.8676374022267322,\n  'bagging_fraction': 0.718409562107019,\n  'bagging_freq': 6,\n  'min_child_samples': 50})\n\nMacro Recall: 0.6666666666666666\nMacro Precision: 0.9814814814814815\nMacro F1-score: 0.7682539682539682\nClassification Report:\n              precision    recall  f1-score   support\n\n    Decrease       1.00      0.50      0.67         8\n   Stay Flat       1.00      0.50      0.67         2\n    Increase       0.94      1.00      0.97        85\n\n    accuracy                           0.95        95\n   macro avg       0.98      0.67      0.77        95\nweighted avg       0.95      0.95      0.94        95\n"

In [ ]:
# import numpy as np
# import optuna
# import lightgbm as lgb
# import pandas as pd
# import os

# class HyperparameterResearch:
#     def __init__(self, research_name, base_params, num_studies, num_trials):
#         self.research_name = research_name
#         self.base_params = base_params
#         self.num_studies = num_studies
#         self.num_trials = num_trials

#         # Ensure research directory exists
#         if not os.path.exists(research_name):
#             os.makedirs(research_name)

#     def adjust_param_distribution(self, df_describe, original_params):
#         new_params = original_params.copy()
#         for param in original_params:
#             param_stats = df_describe[param]
#             if param_stats['max'] >= original_params[param][1]:
#                 new_max = round(param_stats['max'] + param_stats['std'])
#                 new_params[param] = (original_params[param][0], new_max)
#         return new_params

#     def run_study(self, study_number, params):
#         def objective(trial):
#             train_x, valid_x, train_y, valid_y = X_train, X_test, y_train, y_test
#             dtrain = lgb.Dataset(train_x, label=train_y)

#             param = {
#                 "objective": "multiclass",
#                 "metric": "multi_logloss",
#                 "num_class": 3,
#                 "seed": 42,
#                 "deterministic":True,
#                 "device_type": "cpu",
#                 "verbosity": -1,
#                 "is_unbalance": True,
#                 "boosting_type": "gbdt",
#                 "lambda_l1": trial.suggest_float("lambda_l1", params["lambda_l1"][0], params["lambda_l1"][1], log=True),
#                 "lambda_l2": trial.suggest_float("lambda_l2", params["lambda_l2"][0], params["lambda_l2"][1], log=True),
#                 "num_leaves": trial.suggest_int("num_leaves", params["num_leaves"][0], params["num_leaves"][1]),
#                 "feature_fraction": trial.suggest_float("feature_fraction", params["feature_fraction"][0], params["feature_fraction"][1]),
#                 "bagging_fraction": trial.suggest_float("bagging_fraction", params["bagging_fraction"][0], params["bagging_fraction"][1]),
#                 "bagging_freq": trial.suggest_int("bagging_freq", params["bagging_freq"][0], params["bagging_freq"][1]),
#                 "min_child_samples": trial.suggest_int("min_child_samples", params["min_child_samples"][0], params["min_child_samples"][1]),
#             }

#             gbm = lgb.train(param, dtrain)
#             preds = gbm.predict(valid_x)
#             pred_labels = np.rint(preds)
#             logloss = sklearn.metrics.log_loss(valid_y, pred_labels)
#             return logloss

#         study = optuna.create_study(direction="minimize")
#         study.optimize(objective, n_trials=self.num_trials, show_progress_bar=True)

#         trial_results = []
#         for trial in study.trials:
#             trial_data = {
#                 'Trial Number': trial.number,
#                 'Value': trial.value
#             }
#             trial_data.update(trial.params)
#             trial_results.append(trial_data)

#         df_trials = pd.DataFrame(trial_results)
#         study_dir = os.path.join(self.research_name, f'study_{study_number}')
#         if not os.path.exists(study_dir):
#             os.makedirs(study_dir)
#         df_trials.to_csv(os.path.join(study_dir, 'trials.csv'), index=False)

#         return df_trials

#     def run_research(self):
#         params = self.base_params
#         for study_number in range(self.num_studies):
#             df_trials = self.run_study(study_number, params)
#             if study_number < self.num_studies - 1:
#                 df_best = df_trials.sort_values(by='Value').iloc[:int(len(df_trials)/5)]
#                 df_describe = df_best.describe()
#                 params = self.adjust_param_distribution(df_describe, params)

#         print("Research completed and results saved.")

In [ ]:
import numpy as np
import optuna
import lightgbm as lgb
import pandas as pd
import os
import joblib
import json

class HyperparameterResearch:
    def __init__(self, research_name, base_params, num_studies, num_trials):
        self.research_name = research_name
        self.base_params = base_params
        self.num_studies = num_studies
        self.num_trials = num_trials

        # Ensure research directory exists
        if not os.path.exists(research_name):
            os.makedirs(research_name)

    def adjust_param_distribution(self, df_describe, original_params):
        new_params = original_params.copy()
        for param in original_params:
            param_stats = df_describe[param]
            if param_stats['max'] >= original_params[param][1]:
                new_max = round(param_stats['max'] + param_stats['std'])
                new_params[param] = (original_params[param][0], new_max)
        return new_params

    def run_study(self, study_number, params):
        def objective(trial):
            train_x, valid_x, train_y, valid_y = X_train, X_test, y_train, y_test
            dtrain = lgb.Dataset(train_x, label=train_y)

            param = {
                "objective": "multiclass",
                "metric": "multi_logloss",
                "num_class": 3,
                "seed": 42,
                "deterministic":True,
                "device_type": "cpu",
                "verbosity": -1,
                "is_unbalance": True,
                "boosting_type": "gbdt",
                "lambda_l1": trial.suggest_float("lambda_l1", params["lambda_l1"][0], params["lambda_l1"][1], log=True),
                "lambda_l2": trial.suggest_float("lambda_l2", params["lambda_l2"][0], params["lambda_l2"][1], log=True),
                "num_leaves": trial.suggest_int("num_leaves", params["num_leaves"][0], params["num_leaves"][1]),
                "feature_fraction": trial.suggest_float("feature_fraction", params["feature_fraction"][0], params["feature_fraction"][1]),
                "bagging_fraction": trial.suggest_float("bagging_fraction", params["bagging_fraction"][0], params["bagging_fraction"][1]),
                "bagging_freq": trial.suggest_int("bagging_freq", params["bagging_freq"][0], params["bagging_freq"][1]),
                "min_child_samples": trial.suggest_int("min_child_samples", params["min_child_samples"][0], params["min_child_samples"][1]),
            }

            gbm = lgb.train(param, dtrain)
            preds = gbm.predict(valid_x)
            pred_labels = np.rint(preds)
            logloss = log_loss(valid_y, pred_labels)
            return logloss

        study = optuna.create_study(direction="minimize")
        study.optimize(objective, n_trials=self.num_trials, show_progress_bar=True, n_jobs=-1)

        trial_results = []
        for trial in study.trials:
            trial_data = {
                'Trial Number': trial.number,
                'Value': trial.value
            }
            trial_data.update(trial.params)
            trial_results.append(trial_data)

        df_trials = pd.DataFrame(trial_results)
        study_dir = os.path.join(self.research_name, f'study_{study_number}')
        if not os.path.exists(study_dir):
            os.makedirs(study_dir)
        df_trials.to_csv(os.path.join(study_dir, 'trials.csv'), index=False)

        # Save the study
        joblib.dump(study, os.path.join(study_dir, 'study.pkl'))

        # Save the parameter space
        with open(os.path.join(study_dir, 'param_space.json'), 'w') as f:
            json.dump(params, f, indent=4)

        return df_trials

    def run_research(self):
        params = self.base_params
        for study_number in range(self.num_studies):
            df_trials = self.run_study(study_number, params)
            if study_number < self.num_studies - 1:
                df_best = df_trials.sort_values(by='Value').iloc[:int(len(df_trials)/5)]
                df_describe = df_best.describe()
                params = self.adjust_param_distribution(df_describe, params)

        print("Research completed and results saved.")


In [ ]:
# # Define the initial parameter ranges
# l1_start, l1_end = 1e-8, 10.0
# l2_start, l2_end = 1e-8, 10.0
# num_leaves_start, num_leaves_end = 2, 256
# feature_fraction_start, feature_fraction_end = 0.4, 1.0
# bagging_fraction_start, bagging_fraction_end = 0.4, 1.0
# bagging_freq_start, bagging_freq_end = 1, 7
# min_child_samples_start, min_child_samples_end = 5, 100

# base_params = {
#     "lambda_l1": (l1_start, l1_end),
#     "lambda_l2": (l2_start, l2_end),
#     "num_leaves": (num_leaves_start, num_leaves_end),
#     "feature_fraction": (feature_fraction_start, feature_fraction_end),
#     "bagging_fraction": (bagging_fraction_start, bagging_fraction_end),
#     "bagging_freq": (bagging_freq_start, bagging_freq_end),
#     "min_child_samples": (min_child_samples_start, min_child_samples_end),
# }

# # Initialize the research
# research_name = "research_about_cpi_5"
# num_studies = 50
# num_trials = 200

# research = HyperparameterResearch(research_name, base_params, num_studies, num_trials)

# research.run_research()


  0%|          | 0/200 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, accuracy_score, log_loss, classification_report
import lightgbm as lgb

params = best_trial.params

model = lgb.LGBMClassifier(**params, verbose=-1)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)

recall = recall_score(y_test, y_pred, average='macro')
precision = precision_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')
accuracy = accuracy_score(y_test, y_pred)
logloss = log_loss(y_test, y_prob)

print(f"Log Loss: {logloss}")
print(f"Accuracy: {accuracy}")
print(f"Macro Recall: {recall}")
print(f"Macro Precision: {precision}")
print(f"Macro F1-score: {f1}")
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=['0', '1', '2']))

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Log Loss: 0.4450425803154309
Accuracy: 0.9263157894736842
Macro Recall: 0.4583333333333333
Macro Precision: 0.6413043478260869
Macro F1-score: 0.5019688409518918
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.38      0.55         8
           1       0.00      0.00      0.00         2
           2       0.92      1.00      0.96        85

    accuracy                           0.93        95
   macro avg       0.64      0.46      0.50        95
weighted avg       0.91      0.93      0.91        95



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:13